
________________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________________


# Analyse de sentiments en temps réel - Streaming sentiment analysis


________________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________________

## Sujet - Subject

Nous allons récupérer les textes que vous allez taper directement sur vos ordinateurs et nous dirons à l'aide d'un algorithme statistique si votre texte est plutôt positif ou négatif. Le modèle utilisé est celui créé dans le notebook précédent.
> 
*
We are going to collect the text that you have taped on your computer and we will predict the sentiment (positive/negative) of this text. The model is the one which we have built in the previous notebook.
*

## SparkContext

In [1]:
# Si il n'y a pas de SparkContext, il faut le créer / Create a SparkContext if there is not :
#from pyspark import SparkContext # Si il n'existe pas déjà / if it doesn't exist
#sc = SparkContext(appName="PythonStreamingKafka_SentimentAnalysis")

## Importation des libraries - Library import

In [2]:
## Modules pour la liaison entre Kafka et Spark / Packages for linking Kafka et Spark

from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [3]:
## Modules pour le traitement de texte et le modèle / Packages for text formatting and model

# Transformation des données / Data transformation
from pyspark.mllib.linalg import Vectors  
from pyspark.mllib.regression import LabeledPoint

# Pour analyser les résultats / For analyzing the results
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.mllib.evaluation import BinaryClassificationMetrics

# Pour la régression logistique / For logistic regression
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.classification import LogisticRegressionModel

## Chargement du dictionnaire - Loading of the dictionnary

Nous disposons d'un dictionnaire de mots dont chaque mot est une variable du modèle.
> 
*We have a dictionnay of words for which each word is a variable (a feature) of the model.*

In [4]:
liste_mots = sc.textFile("hdfs://ecoles.node1.pro.hupi.loc/user/anthony.laffond/my_liste2600py/*") \
               .map(lambda l: (l.encode("utf-8"),0)).collect()

print("Nombre de mots retenus : %d" %len(liste_mots))
print("Type d'objet : "+str(type(liste_mots)))

print("Apercu :")
pos = liste_mots.index(("texte",0))
liste_mots[(pos-5):(pos+5)]

Nombre de mots retenus : 2668
Type d'objet : <type 'list'>
Apercu :


[('terriblement', 0),
 ('terribles', 0),
 ('territoire', 0),
 ('tete', 0),
 ('teule', 0),
 ('texte', 0),
 ('textes', 0),
 ('thanatonautes', 0),
 ('the', 0),
 ('theatre', 0)]

## Mise en forme du texte - Text formatting

Nous devons d'abord nettoyer le texte des accents, caractères spéciaux et autres.
> 
*First of all, we have to clean the text from any accents, specials characters and others.*

In [5]:
def nettoyage_texte(text) :
    texte = text.replace('À', 'A').replace('Á', 'A').replace('Â', 'A').replace('Ã', 'A') \
    .replace('È', 'E').replace('É', 'E').replace('Ê', 'E').replace('Ë', 'E') \
    .replace('Í', 'I').replace('Ì', 'I').replace('Î', 'I').replace('Ï', 'I') \
    .replace('Ù', 'U').replace('Ú', 'U').replace('Û', 'U').replace('Ü', 'U') \
    .replace('Ò', 'O').replace('Ó', 'O').replace('Ô', 'O').replace('Õ', 'O') \
    .replace('Ö', 'O').replace('Ñ', 'N').replace('Ç', 'C').replace('ª', 'A') \
    .replace('º', 'O').replace('§', 'S').replace('³', '3').replace('²', '2') \
    .replace('¹', '1').replace('à', 'a').replace('á', 'a').replace('â', 'a') \
    .replace('ã', 'a').replace('ä', 'a').replace('è', 'e').replace('é', 'e') \
    .replace('ê', 'e').replace('ë', 'e').replace('í', 'i').replace('ì', 'i') \
    .replace('î', 'i').replace('ï', 'i').replace('ù', 'u').replace('ú', 'u') \
    .replace('û', 'u').replace('ü', 'u').replace('ò', 'o').replace('ó', 'o') \
    .replace('ô', 'o').replace('õ', 'o').replace('ö', 'o').replace('ñ', 'n') \
    .replace('Ä', 'A').replace('ç', 'c') \
    .replace("!"," ").replace("."," ").replace("?"," ").replace(","," ") \
    .replace(";"," ").replace(":"," ").replace("/"," ").replace("+"," ") \
    .replace("%"," ").replace("("," ").replace(")"," ").replace("["," ") \
    .replace("]"," ").replace("&"," ").replace("`"," ").replace("*"," ") \
    .replace("$"," ").replace("«"," ").replace("»"," ").replace("'"," ") \
    .replace("_"," ").replace("\t"," ").replace("|"," ").replace("\""," ") \
    .replace("0"," ").replace("1"," ").replace("2"," ").replace("3"," ") \
    .replace("4"," ").replace("5"," ").replace("6"," ").replace("7"," ") \
    .replace("8"," ").replace("9"," ") \
    .replace("!"," ").replace("."," ").replace("?"," ").replace(","," ") \
    .replace(";"," ").replace(":"," ").replace("/"," ").replace("+"," ") \
    .replace("%"," ").replace("("," ").replace(")"," ").replace("["," ") \
    .replace("]"," ").replace("&"," ").replace("`"," ").replace("*"," ") \
    .replace("$"," ").replace("«"," ").replace("»"," ").replace("'"," ") \
    .replace("_"," ").replace("\t"," ").replace("|"," ").replace("\""," ") \
    .replace(" -"," ").replace("- "," ").replace("--"," ").replace(" - "," ") \
    .lower().strip().split()
    return texte

texte = nettoyage_texte("Ëxémplè : J'äï$ réùssî à néttöyer- ce texte et re- néttöyer môn 1 téxtè !")
print(texte)

['exemple', 'j', 'ai', 'reussi', 'a', 'nettoyer', 'ce', 'texte', 'et', 're', 'nettoyer', 'mon', 'texte']


Ensuite, il faut mettre en forme le texte pour que le modèle puisse lire les données
> *Then, we have to format the text so that the model can read the data*

In [6]:
def mise_en_forme(liste_mots, texte, sentiment = 1):
    data = zip(texte, [texte.count(w) for w in texte]) # Contage des mots du texte (effectifs)
    data = list(set(data))  # Retrait des doublons
    data = [w for w in data if (w[0],0) in liste_mots] # Filtre les mots retenus précédemment (dans liste_mots)
    # Ajout des mots retenus qui ne sont pas présents dans la phrase sous la forme: (mot,0)
    data = data +[ w for w in liste_mots if w[0] not in [x[0] for x in data] ]
    data = sorted(data, key=lambda l: l[0])  # On trie par ordre alphabétique des mots
    data = LabeledPoint( sentiment ,Vectors.dense([w[1] for w in data]) )
    return data
final = mise_en_forme(liste_mots,texte)
print("Sentiment attribué (par défaut) : "+ str(final.label))
print("Variables (mots) : ")
print(final.features[(pos-5):(pos+5)])

# Dans cet exemple, nous avons : / In this example, we have :
# [ 0.  0.  0.  0.  0.  2.  0.  0.  0.  0.] 
# Le 2 correspond au mot "texte" qui apparait 2 fois dans la phrase / the "2" is the number of occurrence of "texte"

Sentiment attribué (par défaut) : 1.0
Variables (mots) : 
[ 0.  0.  0.  0.  0.  2.  0.  0.  0.  0.]


## Chargement du modèle - Loading the model

In [7]:
model = LogisticRegressionModel.load(sc, "hdfs://ecoles.node1.pro.hupi.loc/user/anthony.laffond/model_reglog2600py")

In [8]:
# Fonction de conversion du score / Conversion of score function
def my_pred(score):
    if(score>=0.8):
        val = u"Positif"
    elif(score>=0.6):
        val = u"Moyennement positif"
    elif(score>=0.4):
        val = u"Neutre"
    elif(score>=0.2):
        val = u"Moyennement negatif"
    else:
        val = u"Negatif"
    return val

In [9]:
print(texte)
score = model.predict(final.features)
print("Score attribué : " + str(score))
print("Sentiment attribué : " + str(my_pred(score)) )

['exemple', 'j', 'ai', 'reussi', 'a', 'nettoyer', 'ce', 'texte', 'et', 're', 'nettoyer', 'mon', 'texte']
Score attribué : 1
Sentiment attribué : Positif


In [11]:
def sentiment_analysis(texte,liste_mots,model):
    texte = nettoyage_texte(texte)
    texte = mise_en_forme(liste_mots, texte)
    pred = model.predict(texte.features)
    return pred

phrase = "Ëxémplè : J'äï$ réùssî à Faîrë de l'ànalise de SêntIment !"
print(phrase)
print("Score prédit : " + str(sentiment_analysis(phrase,liste_mots,model) ) )
print("Sentiment prédit : " + my_pred(sentiment_analysis(phrase,liste_mots,model) ).encode("utf8") )

Ëxémplè : J'äï$ réùssî à Faîrë de l'ànalise de SêntIment !
Score prédit : 1
Sentiment prédit : Positif


## Liaison avec Kafka - Link with Kafka

### Initialisation - Initialization

In [18]:
intervalle = 10 # Fenêtre de x secondes / x seconds window

ssc = StreamingContext(sc, intervalle)

zkQuorum = "ecoles.node1.pro.hupi.loc:2181"

topic = {"ecoles_anthony": 1}

identifiant = "Anthony"  # A modifier pour chaque personne / modify for each user

streamdata = KafkaUtils.createStream(ssc, zkQuorum, identifiant, topic)

### Traitement - Treatment

In [19]:
# Sauvegarder les messages en .txt / Save messages as .txt files 
# fichier = streamdata.map(lambda l: l[1])
# fichier.saveAsTextFiles("hdfs://ecoles.node1.pro.hupi.loc/user/anthony.laffond/Streaming/fichier")

In [20]:
# Fonction d'affichage / Display fonction
def get_output(rdd):
    li = rdd.collect()
    for x in li:
        print(str(x[0].encode("utf8")) + " --> Score = " + str(x[1]) + " --> " + str(x[2]))
    print("\n")

In [21]:
intro = streamdata.count().map(lambda l : "\n Nombre de textes : "+str(l))
intro.pprint()

fichier = streamdata.map(lambda l: l[1]) \
                    .map(lambda rdd : rdd[(rdd.find("\"log\":")+len("\"log\":")):rdd.find(",\"action\"")]) \
                    .map(lambda l : ( l , \
                                (sentiment_analysis(l.encode("utf8"),liste_mots,model)*1.0), \
                                str(my_pred(sentiment_analysis(l.encode("utf8"),liste_mots,model))) \
                                    ) )
fichier.foreachRDD(lambda w : get_output(w))

### Lancement du programme - Program's launching

In [22]:
ssc.start()
ssc.awaitTermination(intervalle)

-------------------------------------------
Time: 2016-05-27 17:52:50
-------------------------------------------



-------------------------------------------
Time: 2016-05-27 17:53:00
-------------------------------------------

 Nombre de textes : 1

"l'INSA est une école géniale !" --> Score = 1.0 --> Positif




In [23]:
ssc.stop(stopSparkContext=False)